#  Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
from scipy.spatial.distance import euclidean

In [3]:
data = pd.read_table('checkins.dat', sep='|')[1:]

In [4]:
columns = [ str(i.strip()) for i in data.columns]
columns

['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']

In [5]:
data.columns

Index(['   id    ', ' user_id ', ' venue_id ', '     latitude      ',
       '     longitude     ', '     created_at      '],
      dtype='object')

In [6]:
df = pd.DataFrame(data)

In [7]:
df.columns = columns

In [8]:
def replace(x):
    if x == '                   ':
        return np.nan
    else:
        return x

In [9]:
for column in df:
    df[column] = df[column].apply(replace)

In [10]:
df = df.dropna()

In [11]:
train_df = df.drop('created_at', axis=1)[:100000]

In [12]:
X = train_df[['latitude', 'longitude']]
X

,latitude,longitude
2,38.8951118,-77.0363658
4,33.800745,-84.41052
8,45.5234515,-122.6762071
10,40.764462,-111.904565
11,33.4483771,-112.0740373
...,...,...
233789,33.575,-117.7255556
233790,37.629349,-122.400087
233794,29.7628844,-95.3830615
233798,32.802955,-96.769923


In [14]:
clustering = MeanShift(bandwidth=0.1).fit(X)

In [29]:
val, count = np.unique(clustering.labels_, return_counts=True)

In [32]:
labels = [x for x in val if count[x] > 15]

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [34]:
offices = [(33.751277, -118.188740),(25.867736, -80.324116),
           (51.503016, -0.075479), (52.378894, 4.885084),
           (39.366487, 117.036146),(-33.868457, 151.205134),]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [49]:
ans = []
for of in offices:
    dif = 999999
    for i in labels:
        if euclidean(of, clustering.cluster_centers_[i]) < dif:
            dif = euclidean(of, clustering.cluster_centers_[i])
            center = clustering.cluster_centers_[i][0],clustering.cluster_centers_[i][1]
    ans.append((center[0], center[1], dif))


In [47]:
ans.sort(key=lambda x: x[2])

In [50]:
ans

[(33.8098779552631, -118.14892380690813, 0.07084773242719973),
 (25.8456722642857, -80.31889059642857, 0.022674066158385495),
 (51.502991260887086, -0.12553728870967767, 0.05005829482278787),
 (52.37296399032261, 4.892317222580647, 0.009353316185992226),
 (31.230393000000017, 121.473704, 9.267575010767363),
 (-33.86063042857143, 151.20477592857145, 0.007834758163107856)]